What this service does: Generates a summary of a document.

Summarization is a combination of generative Large Language models and task-optimized encoder models. The service provides summarization solutions for plain texts, conversations, and native documents. 

Supported Input:
- Text summarization only accepts plain text blocks. 
- Conversation summarization accepts conversational input, including various speech audio signals. 
- Native document summarization accepts documents in their native formats, such as Word, PDF, or plain text. 

Prerequisites:
Make sure you have a Language resource deployed on Azure. If that's the case, you can fetch your credentials -- Azure Language endpoint and key -- on Azure Portal. 


In [4]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
from dotenv import load_dotenv

load_dotenv()

endpoint = os.getenv("AZURE_LANGUAGE_ENDPOINT")
key = os.getenv("AZURE_LANGUAGE_KEY")

text_analytics_client = TextAnalyticsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(key),
)

The input is passed as a list of documents. Each document can be passed as a string in the list or as a list of DetectLanguageInput or TextDocumentInput, or a dict-like representation of the object.

In [5]:
document = [
        "At Microsoft, we have been on a quest to advance AI beyond existing techniques, by taking a more holistic, "
        "human-centric approach to learning and understanding. As Chief Technology Officer of Azure AI Cognitive "
        "Services, I have been working with a team of amazing scientists and engineers to turn this quest into a "
        "reality. In my role, I enjoy a unique perspective in viewing the relationship among three attributes of "
        "human cognition: monolingual text (X), audio or visual sensory signals, (Y) and multilingual (Z). At the "
        "intersection of all three, there's magic-what we call XYZ-code as illustrated in Figure 1-a joint "
        "representation to create more powerful AI that can speak, hear, see, and understand humans better. "
        "We believe XYZ-code will enable us to fulfill our long-term vision: cross-domain transfer learning, "
        "spanning modalities and languages. The goal is to have pretrained models that can jointly learn "
        "representations to support a broad range of downstream AI tasks, much in the way humans do today. "
        "Over the past five years, we have achieved human performance on benchmarks in conversational speech "
        "recognition, machine translation, conversational question answering, machine reading comprehension, "
        "and image captioning. These five breakthroughs provided us with strong signals toward our more ambitious "
        "aspiration to produce a leap in AI capabilities, achieving multisensory and multilingual learning that "
        "is closer in line with how humans learn and understand. I believe the joint XYZ-code is a foundational "
        "component of this aspiration, if grounded with external knowledge sources in the downstream AI tasks."
    ]

An alternative way to pass documents is shown in the cell bellow:

In [8]:
documents_dict = [
        {"id": "0", "country_hint": "US", "text": "I had the best day of my life. I decided to go sky-diving and it made me appreciate my whole life so much more. I developed a deep-connection with my instructor as well."},
        {"id": "1", "country_hint": "GB",
         "text": "This was a waste of my time. The speaker put me to sleep."},
        {"id": "2", "country_hint": "MX", "text": "No tengo dinero ni nada que dar..."},
        {"id": "3", "country_hint": "FR",
         "text": "L'hôtel n'était pas très confortable. L'éclairage était trop sombre."}
    ]

There are 2 available mathods for summarization: extractive and abstractive. Both methods are part of the TextAnalyticsClient class. 

The first one returns the most salient sentences from the original document that correspond to the main idea of the text. The second method returns a summary of the whole document using phrases and synonyms that were not necessarily present in the original document.

Run the cell bellow to get an extractive summary of the document.

In [7]:
poller = text_analytics_client.begin_extract_summary(document)
extract_summary_results = poller.result()
for result in extract_summary_results:
    if result.kind == "ExtractiveSummarization":
        print("Summary extracted: \n{}".format(
             " ".join([sentence.text for sentence in result.sentences]))
        )
    elif result.is_error is True:
        print("...Is an error with code '{}' and message '{}'".format(
            result.error.code, result.error.message
        ))

Summary extracted: 
At the intersection of all three, there's magic-what we call XYZ-code as illustrated in Figure 1-a joint representation to create more powerful AI that can speak, hear, see, and understand humans better. We believe XYZ-code will enable us to fulfill our long-term vision: cross-domain transfer learning, spanning modalities and languages. The goal is to have pretrained models that can jointly learn representations to support a broad range of downstream AI tasks, much in the way humans do today.


Run the cell bellow to get an abstractive summary of the document.

In [6]:
poller = text_analytics_client.begin_abstract_summary(document)

abstract_summary_results = poller.result()

for result in abstract_summary_results:
    if result.kind == "AbstractiveSummarization":
        print("Summaries abstracted:")
        [print(f"{summary.text}\n") for summary in result.summaries]
    elif result.is_error is True:
        print("...Is an error with code '{}' and message '{}'".format(
            result.error.code, result.error.message
        ))


Summaries abstracted:
The Chief Technology Officer of Azure AI Cognitive Services discusses Microsoft's commitment to advancing AI by integrating monolingual text, audio or visual signals, and multilingual capabilities, termed as the XYZ-code. This approach aims to create AI that can better understand humans across different domains and languages. Through their efforts, Microsoft has achieved human-level performance on key benchmarks in speech recognition, machine translation, conversational question answering, reading comprehension, and image captioning. The ultimate goal is to develop pretrained models that can learn from multiple modalities and languages, akin to human learning, and incorporate external knowledge sources for downstream AI tasks. This progress is seen as a stepping stone towards a significant leap in AI capabilities, with a focus on multisensory and multilingual learning. The XYZ-code is central to this vision, promising a more holistic and human-centric AI.



Run the cell below to get an abstract summary of the document passed in the alternative format (dict-like representation)

In [12]:
poller = text_analytics_client.begin_abstract_summary(documents_dict)

abstract_summary_results = poller.result()

for result in abstract_summary_results:
    if result.kind == "AbstractiveSummarization":
        print("Summaries abstracted:")
        [print(f"{summary.text}\n") for summary in result.summaries]
    elif result.is_error is True:
        print("...Is an error with code '{}' and message '{}'".format(
            result.error.code, result.error.message
        ))


Summaries abstracted:
The individual experienced an exceptionally positive day, marked by the decision to engage in sky-diving, an activity that profoundly impacted their perspective on life. Through this exhilarating experience, they gained a heightened appreciation for their life' extrinsic events and emotions. A significant aspect of their day was the development of a meaningful connection with their sky-diving instructor, adding depth to the overall experience. This transformative encounter underscored the day's unforgettable nature and its lasting influence on the person's outlook. The summary encapsulates the core elements of joy, life appreciation, and interpersonal connection conveyed in the original document, presenting a coherent and non-redundant account of the experience.

Summaries abstracted:
The individual expressing frustration felt that their time was squandered during an event or presentation they attended. The primary complaint was that the speaker's delivery was sub